# 模块二：通过地图可视化增强地理空间洞察力

**目标**：介绍Folium，这是一个Python库，可轻松在交互式Leaflet地图上可视化数据。

- 创建基本的交互式地图。
- 绘制点数据（例如，城市位置）。
- 创建分级统计图（例如，区域统计数据）。
**要使用的库**：pandas、Folium

In [9]:
#-------------------------------------------------------------------------------
# SETUP
#-------------------------------------------------------------------------------
import pandas as pd
import folium

print("Libraries imported successfully!")

Libraries imported successfully!


## Folium简介

Folium 借助Python在数据处理方面的优势以及Leaflet.js在地图绘制方面的优势。它使你能够创建交互式地图，这些地图可以嵌入到Jupyter笔记本中，也可以保存为HTML文件。

In [2]:
#-------------------------------------------------------------------------------
# Creating a Base Map
#-------------------------------------------------------------------------------
# Create a map centered around a general location, e.g., Europe
map_base = folium.Map(location=[1, 2], zoom_start=4)
print("Base map created. Displaying map (renders directly in Jupyter):")
map_base # Display the map

Base map created. Displaying map (renders directly in Jupyter):


**不同的Tile Layer**：

Folium支持多种Tile Layer。常见的包括：
- “OpenStreetMap”（默认）
- “CartoDB positron” / “CartoDB dark_matter”
- “Stamen Terrain”、“Stamen Toner”、“Stamen Watercolor”

In [4]:
# --- Showing different tiles ---
map_tiles_example = folium.Map(location=[40.7128, -74.0060], zoom_start=10, tiles='CartoDB positron')
folium.Marker(
    location=[40.7128, -74.0060],
    popup='New York City',
    tooltip='Click for NYC!'
).add_to(map_tiles_example)
print("\nMap with 'CartoDB positron' tiles:")
map_tiles_example


Map with 'CartoDB positron' tiles:


## 点数据的标记绘制

我们可以在地图上添加标记来表示特定位置，如城市、研究地点或事件发生地。

In [5]:
#-------------------------------------------------------------------------------
# Hands-on: Plotting World Cities (Code Cells)
#-------------------------------------------------------------------------------
# --- Load City Data ---
cities_df = pd.read_csv('worldcities.csv')
print("Sample cities data:")
print(cities_df)

Sample cities data:
          city city_ascii      lat       lon       country iso2 iso3  \
0        Tokyo      Tokyo  35.6870  139.7495         Japan   JP  JPN   
1      Jakarta    Jakarta  -6.1750  106.8275     Indonesia   ID  IDN   
2        Delhi      Delhi  28.6100   77.2300         India   IN  IND   
3    Guangzhou  Guangzhou  23.1300  113.2600         China   CN  CHN   
4       Mumbai     Mumbai  19.0761   72.8775         India   IN  IND   
..         ...        ...      ...       ...           ...  ...  ...   
994    Yanggok    Yanggok  37.6333  127.2167  Korea, South   KR  KOR   
995    Zhongba    Zhongba  31.7710  104.7550         China   CN  CHN   
996     Osogbo     Osogbo   7.7667    4.5667       Nigeria   NG  NGA   
997     Sizhan     Sizhan  38.9846  106.3828         China   CN  CHN   
998      Suohe      Suohe  34.7833  113.3500         China   CN  CHN   

      admin_name  capital  population          id  
0          Tōkyō  primary    37785000  1392685764  
1        Ja

In [6]:
# --- Create Map and Add Markers ---
map_cities = folium.Map(location=[cities_df['lat'].mean(), cities_df['lon'].mean()], zoom_start=2)

for index, row in cities_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['city']}<br>Population: {row['population']:,}", # Formatted popup
        tooltip=row['city'],
        icon=folium.Icon(color='blue', icon='info-sign') # Example custom icon
    ).add_to(map_cities)

print("\nMap with city markers:")
map_cities


Map with city markers:


## 创建分级统计图

分级统计图使用颜色深浅来表示预先定义的地理区域（如国家、州、县）的数据值。
通常你需要：

1. 一个定义区域边界的GeoJSON/TopoJSON文件。
2. 一个包含这些区域对应值的数据集（如Pandas数据框）。

In [7]:
#-------------------------------------------------------------------------------
# Hands-on: US States Population (Example) (Code Cells)
#-------------------------------------------------------------------------------
# --- Prepare Data ---
# GeoJSON for US States (often available online, e.g., from Folium examples or public sources)
# For simplicity, let's assume a GeoJSON file `us-states.json` is in the same directory or provide a direct URL.
# If using a local file: us_states_geojson_path = 'us_states.json'
# Make sure this file exists or adjust the path.
us_states_geojson_path = "us_states.json"

unemployment_df = pd.read_csv('us_unemployment_oct_2012.csv')
print("US States unemployment data:")
print(unemployment_df)

US States unemployment data:
   State  Unemployment
0     AL           7.1
1     AK           6.8
2     AZ           8.1
3     AR           7.2
4     CA          10.1
5     CO           7.7
6     CT           8.4
7     DE           7.1
8     FL           8.2
9     GA           8.8
10    HI           5.4
11    ID           6.6
12    IL           8.8
13    IN           8.4
14    IA           5.1
15    KS           5.6
16    KY           8.1
17    LA           5.9
18    ME           7.2
19    MD           6.8
20    MA           6.7
21    MI           9.1
22    MN           5.6
23    MS           9.1
24    MO           6.7
25    MT           5.8
26    NE           3.9
27    NV          10.3
28    NH           5.7
29    NJ           9.6
30    NM           6.8
31    NY           8.4
32    NC           9.4
33    ND           3.2
34    OH           6.9
35    OK           5.2
36    OR           8.5
37    PA           8.0
38    RI          10.1
39    SC           8.8
40    SD           4.4
41   

In [8]:
# --- Create Choropleth Map ---
# Create a base map centered on the US
map_choropleth = folium.Map(location=[43, -100], zoom_start=4)

# Add the Choropleth layer
# The `key_on` parameter needs to match the property in your GeoJSON that identifies the states.
# Common ones are 'feature.id' or 'feature.properties.name' or 'feature.properties.STUSPS'
# You might need to inspect your GeoJSON file to find the correct key.
# For the example us_states.json from folium, 'feature.id' often works for 2-letter state codes.
try:
    folium.Choropleth(
        geo_data='us_states.json', # Path or URL to GeoJSON
        name='choropleth',
        data=unemployment_df,
        columns=['State', 'Unemployment'], # DataFrame columns: 1st for key, 2nd for value
        key_on='feature.id',      # Path to the field in GeoJSON to bind data on (e.g., 'feature.id' or 'feature.properties.name')
        fill_color='YlGnBu',      # Color scheme (e.g., 'YlGn', 'PuRd', 'BuPu')
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Unemployment by State',
        highlight=True # Highlight feature on mouseover
    ).add_to(map_choropleth)

    folium.LayerControl().add_to(map_choropleth) # If you have multiple layers

    print("\nChoropleth map created. Displaying map:")
    display(map_choropleth)
except Exception as e:
    print(f"\nCould not create choropleth map. Error: {e}")
    print("Please ensure the GeoJSON URL is correct and accessible, and the `key_on` parameter matches your GeoJSON structure.")
    print("The `key_on` for the Folium example 'us-states.json' is typically 'feature.id' for 2-letter state codes.")


Choropleth map created. Displaying map:


## 应用

地理空间可视化在以下方面功能强大：

- **研究**：
    - 绘制研究地点、采样位置或物种分布地图。
    - 可视化人口统计数据、选举结果或公共卫生模式。
    - 分析空间趋势和差异。
- **教学**：
    - 赋予地理、历史和社会研究以活力。
    - 阐释环境变化或历史迁徙。
    - 为学生创建交互式学习工具以探索空间数据。


## 结论

我们学习了使用Folium创建交互式地图的基础知识，包括绘制点数据和生成分级统计图。这些技能对于任何涉及地理参考数据的工作来说都是非常宝贵的。

## 练习

`car_crashes.csv`是美国各州的交通事故发生率统计，试用地理位置的可视化对各种原因进行可视化。